In [1]:
"""Build initial dataset and write it to S3."""
import pandas as pd

with open("../data/tracks.txt") as fin:
    tracks = [list(map(str.strip, line.split("\t"))) for line in fin.readlines()]
    
tracks_df = pd.DataFrame(tracks, columns=["genre", "artist", "name"])

In [2]:
tracks_df.sample(5)

,genre,artist,name
19795,latino,Greg Giraldo,Greg Giraldo
7873,drum-and-bass,Heavy Rotation,Roni Size
4352,children,The Mice Go Marching,Hap Palmer
21250,minimal-techno,Ghetto Platypus,John Tejada
8641,edm,Down Time,Jr. Gone Wild
